In [91]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle
import time
from bs4 import BeautifulSoup
import random

In [80]:
def pickle_file(obj, path):
    file = open(path, 'wb')
    pickle.dump(obj, file)
    file.close()

def load_pickle(path):
    file = open(path, 'rb')
    obj = pickle.load(file)
    file.close()
    return obj

### Start Webdriver

In [2]:
driver_path = './WebDriver/chromedriver'
driver = webdriver.Chrome(driver_path)

#   options = ChromeOptions()
#   driver = webdriver.Chrome(driver_path,options=options)

/tmp/ipykernel_20036/2995515328.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


### Load Webpage

In [3]:
driver.get('https://www.nasdaq.com/market-activity/ipos')

#### Navigate to old date

### Build List of Dictionaries From Table

In [88]:
def get_starting_page(driver, starting_date = '01/1995'):
    #Open Date 
    date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/button'))
        ).click()

    #Set Date Picker Date
    date_picker = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/input'))
        )
    driver.execute_script(f"arguments[0].value = '{starting_date}';", date_picker)     
        # .setAttribute('value', '01/1999')

    #Navigate to New Page
    apply_date_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div/button[2]'))
        ).click()

def generate_data(driver):
    table = driver.find_element(By.XPATH,
        '/html/body/div[3]/div/main/div[2]/div[2]/div[2]/div/div[2]/div/div[3]/div[5]/div[1]/div[1]/div/table')
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    
    url_base = 'https://www.nasdaq.com'
    table_data = []
    # print(soup)
    # if soup.find('th').text == '':
    #     return []

    rows = soup.find_all('tr',{'class':'market-calendar-table__row'})
    for row in rows:
        row_data = {}

        th = row.find('th', {'role':'cell'})
        ticker = th.findChild().text
        if not ticker:
            continue

        row_data[th['data-column']] = ticker
        row_data['link'] = url_base + row.find('a')['href']

        cells = row.find_all('td',{'role':'cell'})        
        for cell in cells:
            row_data[cell['data-column']] = cell.findChild().text
        table_data.append(row_data)
    return table_data

# generate_data(driver)

def get_next_calendar_page(driver):
    next_scroll = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, ".//button[@class='time-belt__next']"))
        ).click()

    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
        By.XPATH, ".//button[@class='time-belt__item']"))
        ).click()

# get_next_calendar_page(driver)


In [86]:
def main_loop(pickup_file = False, starting_date= '01/1995', delay = 10):
    if pickup_file:
        data = load_pickle(pickup_file)
    else:
        data = []

    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    
    driver_path = './WebDriver/chromedriver'
    driver = webdriver.Chrome(driver_path, options=chrome_options)
    print('Started Driver')

    time.sleep(delay)

    driver.get('https://www.nasdaq.com/market-activity/ipos')
    print('Loaded Initial Page')

    time.sleep(delay)

    get_starting_page(driver, starting_date=starting_date)
    print('Loaded Starting Page')

    date_year = 0
    count = 0
    while date_year <= 2022:
        count+=1
        date_year = int(driver.find_element(By.XPATH, ".//button[@class='time-belt__item']").get_attribute('data-year'))

        data += generate_data(driver)
        pickle_file(data, f'Data/nasdaq_checkpoint_{date_year}')
        print(f'Checkpoint-{date_year}\t(Total: {count};\t Len: {len(data)})',end='\r', flush=True)
        time.sleep(random.randint(10,30))

        get_next_calendar_page(driver)
    
    print('\nAll Done')
    return data

data = main_loop(starting_date='01/1995')



/tmp/ipykernel_20036/1637811262.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options=chrome_options)
/tmp/ipykernel_20036/1637811262.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options=chrome_options)


Started Driver
Loaded Initial Page
Loaded Starting Page
Checkpoint 2022(1)
All Done
[{'proposedTickerSymbol': 'SVRE', 'link': 'https://www.nasdaq.com/market-activity/ipos/overview?dealId=1205111-102356', 'companyName': 'SaverOne 2014 Ltd.', 'proposedExchange': 'NASDAQ Capital', 'proposedSharePrice': '5.80', 'sharesOffered': '2,241,379', 'expectedPriceDate': '06/03/2022', 'dollarValueOfSharesOffered': '$14,949,998.80'}]
